In [10]:
import pandas as pd
import numpy as np
import glob
import matplotlib as mpl
import matplotlib.pyplot as plt 
import os
#import more_itertools as mit

In [11]:

FilePathList = "/Users/homefolder/Documents/RapfLab/code/MStest" #sets filepath for initial file/mass list. May not be necessary to keep this way
                                                            #but sets path for list 
FileType="csv" #sets filetype
MassNumbersList=[] #creates an empty list which will be filled by the numbers in the mass list file
os. chdir(FilePathList) #sets working directory to FilePathList
CurrFile = pd.read_csv("NewIncompMassList.csv") #reads the file masslist.csv from the filepath --> since we only want to read one file was easier to just specify file
#print(CurrFile)
MassNumberNames=[]
#######################################


for i in (range(len(CurrFile))):
    MassNumbersList.append(CurrFile['mz_list'][i]) #appends value number i from masslist.csv in the colum mz_list
    MassNumberNames.append(CurrFile['Species'][i])
print(len(MassNumberNames))
print(len(MassNumbersList))

39
39


In [12]:
FilePathList = "/Users/homefolder/Documents/RapfLab/code/MStest/Processed"
FileType="csv" #sets filetype
os. chdir(FilePathList)
Blanks=[]
CurrFile = pd.read_csv("stdev_blank.csv")
for i in (range(len(CurrFile))):
    Blanks.append(CurrFile['standard deviation'][i])

print(Blanks)

[197.17328238937085, 636.6911765748953, 715.894022649642, 1590.3473001745022, 6.977268679941483, 16834.354356440483, 32970.79355009917, 68.25301013103741, 692.4915308970133, 5.233895395449892, 0.8702889089560607, 8.061556873824413, 70.16283857307546, 70.16283857307546, 966.9888788261418, 3208.765461670305, 37.89142501938961, 1240.088002907507, 16399.242760863042, 2325.35400390625, 31.47558784484865, 19677.02084350586, 6230.4307861328125, 50169.11395263672, 1158.816833496095, 118.949768066405, 2.474278211593605, 274.2600293159485, 4581.94384765625, 52040.462158203125, 12.797760128974923, 1134.686767578125, 3.07194972038269, 367.096130371094, 0.0, 0.1681166291236879, 2.1105384059151, 0.0, 1.4764682228005]


In [17]:

FilePath = "/Users/homefolder/Documents/RapfLab/code/MassSpecData/210711MSCHR/MS" #Second file path for where the actual files are located
os. chdir(FilePath) #updates file path to the folder with actual files for processing
AllFiles=pd.DataFrame()
FileType="csv" #what filetype are the files. Will probably be csv
files = sorted(glob.glob("*"+FileType)) #gets all files of chosen filetype
num_files = len(files) #how many files are you reading in
OutFolder = "Processed" #folder where processed files will be saved
outname = "" #part of file name. Will be used later
##############################
print(files)
ErrorRange = 0.00005 #error range. Will be +- to get range and see if values fit in range. 25 ppm error range     
mzlistRep=[]



for entry in range(num_files): #goes through all files
    name = files[entry] 
    CurrFile = pd.read_csv(files[entry],skiprows=1) #reads files
    FullSplit = name.split(".") #gets all parts of the file name
    NameList=(str(FullSplit[0]))
    mzlistRep=[]
    ProcessedX=[] #values for X(Thompsons) will be saved here
    ProcessedY=[] #values for Y(Counts) will be saved here
    PPM=[] #PPM values will be saved here
    names=[]
    BlankRep=[]
    Detected=[]
    #gets the third part of the name (0,1,[2]). Adds it to the name list
    for mass in range(len(MassNumbersList)): #goes through each mast in the MassesList
        MassHigh=((MassNumbersList[mass])+(MassNumbersList[mass]*ErrorRange)) #upper range of Mass
        MassLow=((MassNumbersList[mass])-(MassNumbersList[mass]*ErrorRange)) #lower range of Mass
        fixed=CurrFile.drop(columns="#Point") #drops point column
        fit=fixed["X(Thompsons)"].between(MassLow, MassHigh, inclusive = True) #checks if value for X(Thompson) fits between mass low and mass high
        ####Part two of function
        for i in range(len(fit)): #checks if values in fit are true or false
             if fit[i]== True: #if True then appends values
                    ProcessedX.append(fixed["X(Thompsons)"][i])
                    ProcessedY.append(fixed["Y(Counts)"][i])
                    PPM.append(((abs((fixed["X(Thompsons)"][i])-MassNumbersList[mass]))/MassNumbersList[mass])*1000000)
                    mzlistRep.append(MassNumbersList[mass])
                    names.append(MassNumberNames[mass])
                    BlankRep.append(Blanks[mass])
                    if fixed(["Y(Counts)"][i])<=3*Blanks[mass] and fixed(["Y(Counts)"][i])>=0: 
                        Detected.append("Not Detected")
                    else:
                        Detected.append("Detected")
             
        if sum(fit) == False: #if all values are false and nothing fits then will append a 0. This makes sure you get the correct
                                  # number of values no matter what
                ProcessedX.append(0)  
                ProcessedY.append(0) #appends 0 for both 
                PPM.append(0)
                mzlistRep.append(MassNumbersList[mass])
                names.append(MassNumberNames[mass])  
                BlankRep.append(Blanks[mass])
                Detected.append("Not Detected")
        
 
        IndFile = pd.DataFrame(columns = ["Compound Names","mz_list",NameList+"_mass",NameList+"_count",NameList+"_PPM", "Blank STDev", "Detected"]) #columns mz_list, mass, and count +NameList[i]
        IndFile["Compound Names"]=names
        IndFile["mz_list"]=mzlistRep
        IndFile[NameList+"_mass"]=ProcessedX
        IndFile[NameList+"_count"]=ProcessedY #for this and the line before, it reads the values of processed from i*length
        IndFile[NameList+"_PPM"]=PPM
        IndFile["BlankSTDev"]=BlankRep
        IndFile["Detected"]=Detected
        IndFile.to_csv("/Users/homefolder/Documents/RapfLab/code/MStest/Processed/"+NameList+"_Processed.csv")          #chooses folder
    ####################


# AllFiles[outname+"mz_list"]=MassNumbersList   #Does same as for IndFile but for an All file with all values. Whole function works in similar way
# AllFiles[outname+"Compound Names"]=MassNumberNames
# for i in range(num_files):
#     AllFiles[outname+"_mass"+NameList[i]]=ProcessedX[(i*length):((i+1)*length)]
#     AllFiles[outname+"_count"+NameList[i]]=ProcessedY[(i*length):((i+1)*length)]
#     AllFiles[outname+"_PPM"+NameList[i]]=PPM[(i*length):((i+1)*length)]
        
# AllFiles.to_csv("/Users/timmo/Documents/RapfLab/code/MStest/List/Processed"+"/"+outname+"_AllProcessed.csv")

    

['210611_OICApost_3_MS.csv', '210611_OVApost_3_MS.csv', '210614_OBApost_4_MS.csv', '210701_365_PApost_MS.csv', '210706_270_PPApost_MS.csv', '210706_365_PA_1hr_MS.csv', '210706_365_PApost_MS.csv', '210706_lamp_PPApost_MS.csv', '210707_270_PApost_MS.csv', '210708_270_PPApost_MS.csv', '210708_365_PPApost_MS.csv', '210708_lamp_PPApost_MS.csv', '210711_MeOHBlank_01Pos_MS.csv', '210711_MeOHBlank_01_MS.csv', '210711_MeOHBlank_02_MS.csv', '210711_MeOHBlank_02pos_MS.csv', '210711_MeOHBlank_03_MS.csv', '210711_MeOHBlank_03pos_MS.csv', '210711_MeOHBlank_04_MS.csv', '210711_MeOHBlank_04pos_MS.csv', '210711_MeOHBlank_05_MS.csv', '210711_MeOHBlank_05pos_MS.csv', '210711_MeOHBlank_06_MS.csv', '210711_MeOHBlank_06pos_MS.csv', '210711_MeOHBlank_07_MS.csv', '210711_MeOHBlank_07pos_MS.csv', '210711_MeOHBlank_08_MS.csv', '210711_MeOHBlank_09_MS.csv', '210711_MeOHBlank_10_MS.csv', '210711_MeOHBlank_11_MS.csv', '210711_MeOHBlank_12_MS.csv', '210711_MeOHBlank_13_MS.csv', '210711_MeOHBlank_14_MS.csv', '210711

IndexError: list index out of range